In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
from sklearn import model_selection
from sklearn.metrics import mean_squared_error

In [ ]:
from os.path import join
from google.colab import drive

ROOT = "/content/drive"
drive.mount(ROOT)

In [ ]:
PROJ = "My Drive/Colab Notebooks/AML Workspace/EM_KDE_vs_simple_KDE_imp" # This is a custom path.
PROJECT_PATH = join(ROOT, PROJ)

In [ ]:
from importlib.machinery import SourceFileLoader
utils = SourceFileLoader('utils', join(PROJECT_PATH, 'utils.py')).load_module()
plot = SourceFileLoader('plot', join(PROJECT_PATH, 'plot.py')).load_module()

In [ ]:
from utils import remove_random_value, remove_dim, conditional_expectation, e_step, m_step, \
    calculate_log_likelihood, is_converged
from plot import plot_kde

 Load data

In [ ]:
data = loadmat(join(ROOT, 'My Drive/Colab Notebooks/AML Workspace/faithfull/faithful.mat'))['X']

 Real world data (may make sense to crop end, since it's quite big)

In [ ]:
# data = np.genfromtxt(join(ROOT, 'My Drive/Colab Notebooks/AML Workspace/data/winequality-white.csv'), delimiter=';')[1:,:80]

 Testing with higher dimension data<br>

In [ ]:
# np.random.shuffle(data)
# data = np.concatenate([data, loadmat(join(ROOT, 'My Drive/Colab Notebooks/AML Workspace/faithfull/faithful.mat'))['X']], axis=1)

In [ ]:
raw_data = data  
data = np.array(raw_data[:-10])
[damaged_data, removed_values] = remove_random_value(raw_data[-10:])

In [ ]:
num_data, dim = data.shape

K-fold cross validation

In [ ]:
K = num_data
CV = model_selection.KFold(n_splits=K, shuffle=False)

 Loop until you're happy

In [ ]:
epsilon = 1e-3
sigma = np.eye(dim)
log_likelihood = np.asarray([])
i = 0
while True:
    i += 1
    sigmas = []
    R = np.linalg.cholesky(sigma)
    A = data.dot(np.linalg.inv(R).T)
    for train_index, test_index in CV.split(A):
        # extract training and test set for current CV fold
        a_test = A[test_index, :]
        a_train = A[train_index, :]
        x_test = data[test_index, :]
        x_train = data[train_index, :]

        # E step
        responsibility = e_step(a_test, a_train, R)

        # M step
        sigmas.append(m_step(x_test, x_train, responsibility))
    sigma = np.array(sigmas).sum(axis=1).mean(axis=0)
    R = np.linalg.cholesky(sigma)
    A = data.dot(np.linalg.inv(R).T)
    _log_likelihood = []
    for train_index, test_index in CV.split(A):
        # extract training and test set for current CV fold
        x_train = A[train_index, :]
        x_test = A[test_index, :]
        _log_likelihood.append(calculate_log_likelihood(x_test, x_train, R))
    log_likelihood = np.append(log_likelihood, np.asarray(_log_likelihood).mean())
    if is_converged(log_likelihood, epsilon):
        break

In [ ]:
plt.figure(1)
plt.plot(log_likelihood)
plt.xlabel('Iterations')
plt.ylabel('Log-likelihood')
plt.show()

# 
<br>
#<br>
# sigma = [[4.28747436e-02, 2.92396851e-01, 2.46394066e-04, 1.05465785e-01],<br>
#          [2.92396851e-01, 1.44238149e+01, 4.95674770e-02, -1.75754718e+00],<br>
#          [2.46394066e-04, 4.95674770e-02, 5.51668545e-02, 2.07264980e-01],<br>
#          [1.05465785e-01, -1.75754718e+00, 2.07264980e-01, 1.57786340e+01]]<br>
#<br>

<br>
<br>
sigma = [[0.0322203, 0.0194771],<br>
         [0.0194771, 3.8548159]]<br>
<br>
# 
<br>
imputed_values = []<br>
restored_data = []<br>
for test_data in damaged_data:<br>
    # get index of missing dimension<br>
    missing_dim = [idx for idx, value in enumerate(test_data) if np.isnan(value)][0]<br>
    # remove data of that dimension<br>
    train_data = data<br>
    reduced_train_data = np.delete(train_data, missing_dim, axis=1)<br>
    test_data = np.delete(test_data, missing_dim, axis=0)<br>
    reduced_sigma = remove_dim(sigma, missing_dim)<br>
    # create transformed data<br>
    R = np.linalg.cholesky(reduced_sigma)<br>
    R_inv_T = np.linalg.inv(R).T<br>
    a_train = reduced_train_data.dot(R_inv_T)<br>
    a_test = test_data.dot(R_inv_T)<br>
    responsibility = np.squeeze(e_step(a_test, a_train, R))<br>
    cond_exp = np.array([conditional_expectation(mean, test_data, sigma, missing_dim) for mean in train_data])<br>
    imputed_value = np.sum(np.multiply(cond_exp, responsibility))<br>
    imputed_values.append(imputed_value)<br>
    restored_element = np.insert(test_data, missing_dim, imputed_value)<br>
    restored_data.append(restored_element)<br>
restored_data = np.array(restored_data)<br>
imputed_values = np.array(imputed_values)<br>
divergence = np.abs(removed_values - imputed_values) / removed_values<br>
# mse = mean_squared_error(removed_values, imputed_values)<br>
plt.figure(2)<br>
plt.plot(divergence)<br>
plt.xlabel('Index')<br>
plt.ylabel('Imputation error in %')<br>
plt.show()<br>
plot_kde(data, sigma, 0.1)